In [ ]:
# -*- coding: utf-8 -*-
"""Naive strats.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/18ITgq52qINQNnAxk3gWXMr5SrrSWLx1k

# **Cluster Return Prediction with VAR and Evaluation**
"""

import pandas as pd
import numpy as np
import torch
import time

# !pip install pykeops # Assuming pykeops is installed if KMeans part is used elsewhere
from pykeops.torch import LazyTensor # Not used in the refactored VAR part

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
# import warnings # warnings.warn calls will be removed

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Global variable for KMeans, as in the original snippet (though KMeans itself is not used in this VAR logic)
# use_cuda = torch.cuda.is_available()

# import torch # Redundant
import torch.func as func # For vmap
# from tqdm import tqdm # Redundant

# Device configuration (use the one from the original script)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Redundant

# import torch # Redundant
import torch.nn.functional as F
# from torch import vmap # PyTorch native vmap # Using functorch.vmap
# import functorch as func # Redundant, already imported as func

def KMeans(x, K, Niter=300, verbose=True):
    """
    Implements Lloyd's algorithm with K-Means++ initialization for the Euclidean metric.
    This version avoids using .item().
    """

    start_total_time = time.time()
    N, D = x.shape  # Number of samples, dimension of the ambient space
    use_cuda = x.is_cuda
    device = x.device

    # K-Means++ initialization
    if verbose:
        print("Starting K-Means++ initialization...")
    start_init_time = time.time()

    centroids = torch.empty(K, D, dtype=x.dtype, device=device)

    # Boolean mask to track chosen centroids
    has_been_chosen = torch.zeros(N, dtype=torch.bool, device=device)

    # 1. Choose the first centroid uniformly at random from the data points
    # Ensure reproducibility if desired by setting torch.manual_seed before calling KMeans
    # .squeeze() converts shape (1,) to 0-dim scalar tensor
    first_centroid_idx_tensor = torch.randint(0, N, (1,), device=device).squeeze()
    centroids[0] = x[first_centroid_idx_tensor]
    has_been_chosen[first_centroid_idx_tensor] = True

    # 2. For each subsequent centroid (from 1 to K-1)
    for k_idx in range(1, K):
        current_chosen_centroids = centroids[:k_idx, :]

        dist_sq_to_chosen = torch.cdist(x, current_chosen_centroids)**2
        min_dist_sq = dist_sq_to_chosen.min(dim=1).values # Shape: (N,)

        # Check if all distances are zero (e.g., all points are identical to chosen centroids)
        if torch.isclose(min_dist_sq.sum(), torch.tensor(0.0, device=device, dtype=x.dtype)):
            # Fallback: pick a random point not yet chosen, or if all chosen, allow re-pick.
            if verbose:
                print(f"K-Means++: min_dist_sq sum is ~0 at k_idx={k_idx}. Fallback to random unchosen point.")

            available_indices = torch.where(~has_been_chosen)[0]
            if available_indices.numel() == 0: # All points marked as chosen or no unchosen points left
                if verbose: print(f"Warning: No unchosen points left for K-Means++ fallback. Picking randomly from all points.")
                # Pick randomly from all N points
                next_centroid_idx_tensor = torch.randint(0, N, (1,), device=device).squeeze()
            else:
                rand_idx_in_available = torch.randint(0, available_indices.numel(), (1,), device=device).squeeze()
                next_centroid_idx_tensor = available_indices[rand_idx_in_available]
        else:
            # Choose the next centroid with probability proportional to min_dist_sq (D(x)^2)
            # .squeeze() converts shape (1,) to 0-dim scalar tensor
            next_centroid_idx_tensor = torch.multinomial(min_dist_sq, 1).squeeze()

        centroids[k_idx] = x[next_centroid_idx_tensor]
        has_been_chosen[next_centroid_idx_tensor] = True # Mark as chosen

    c = centroids.clone()  # Initial centroids from K-Means++

    if verbose:
        end_init_time = time.time()
        print(f"K-Means++ initialization finished in {end_init_time - start_init_time:.5f}s.")

    # --- K-means loop (Lloyd's algorithm) ---
    x_i = LazyTensor(x.view(N, 1, D))  # (N, 1, D) samples

    if verbose:
        print("Starting Lloyd's iterations...")
    start_lloyd_time = time.time()

    actual_iters = Niter # Default to Niter
    for i in range(Niter):
        c_j = LazyTensor(c.view(1, K, D))  # (1, K, D) centroids

        # E step: assign points to the closest cluster -------------------------
        # ((x_i - c_j) ** 2) results in a LazyTensor.
        # .sum(-1) on this LazyTensor calls our dummy LazyTensor.sum(), returning a torch.Tensor.
        D_ij_summed = ((x_i - c_j) ** 2).sum(-1)  # (N, K) symbolic squared distances, now a torch.Tensor
        cl = D_ij_summed.argmin(dim=1).long().view(-1)  # Points -> Nearest cluster

        # M step: update the centroids to the normalized cluster average: ------
        c_old = c.clone()

        c.zero_()
        # scatter_add_ expects indices as long tensors. cl is already .long().
        c.scatter_add_(0, cl[:, None].repeat(1, D), x)

        Ncl = torch.bincount(cl, minlength=K).type_as(c).view(K, 1)

        # Handle empty clusters
        for k_loop_idx in range(K): # k_idx is already used for K-Means++ init loop
            if Ncl[k_loop_idx] > 0:
                c[k_loop_idx] /= Ncl[k_loop_idx]
            else:
                c[k_loop_idx] = c_old[k_loop_idx]
                if verbose and i % 10 == 0 :
                    print(f"Iteration {i}: Cluster {k_loop_idx} is empty. Retaining old centroid.")

        # Optional: convergence check
        if torch.allclose(c, c_old, atol=1e-6, rtol=0):
           if verbose: print(f"Converged at iteration {i+1}.")
           actual_iters = i + 1
           break
    else: # If loop finished without break
        actual_iters = Niter


    if verbose:  # Fancy display -----------------------------------------------
        if use_cuda:
            torch.cuda.synchronize()
        end_lloyd_time = time.time()
        end_total_time = time.time()

        print(
            f"\nK-means++ (no .item()) for the Euclidean metric with {N:,} points in dimension {D:,}, K = {K:,}:"
        )
        print(
            f"Initialization time: {end_init_time - start_init_time:.5f}s"
        )
        print(
            "Lloyd's iterations ({}) : {:.5f}s = {:.5f}s/iter".format(
                actual_iters,
                end_lloyd_time - start_lloyd_time,
                (end_lloyd_time - start_lloyd_time) / actual_iters if actual_iters > 0 else 0
            )
        )
        print(f"Total time: {end_total_time - start_total_time:.5f}s")

    return cl # Return cluster assignments and final centroids


def sqrtinvdiag(M):
    return torch.diag(1.0 / torch.max(torch.sqrt(torch.diag(M)), torch.tensor(1e-10)))

def spectral_clustering_laplacian(p, n, k):
    D_p = torch.diag(p.sum(axis=0))
    D_n = torch.diag(n.sum(axis=0))
    d = sqrtinvdiag(D_p)
    matrix = d @ p @ d
    d = sqrtinvdiag(D_n)
    matrix = matrix - (d @ n @ d)
    matrix = torch.eye(matrix.shape[0], device=device) - matrix
    w, v = torch.linalg.eigh(matrix)
    # u = v[:, :k]
    # # normalize the rows to have unit 2-norm
    # t = u / u.norm(dim=1, keepdim=True)
    return KMeans(v/w, k, verbose=False)

def SPONGE(p, n, k, tau):
    D_p = torch.diag(p.sum(axis=0))
    D_n = torch.diag(n.sum(axis=0))

def calculate_pnl_torch(forecast_tensor, actual_tensor):
    # Removed empty tensor checks and inconsistent shape checks
    min_len = min(forecast_tensor.shape[0], actual_tensor.shape[0])
    common_series = min(forecast_tensor.shape[1], actual_tensor.shape[1])

    forecast_aligned = forecast_tensor[:min_len, :common_series]
    actual_aligned = actual_tensor[:min_len, :common_series]

    positions = torch.sign(forecast_aligned)
    pnl_per_period = positions * actual_aligned
    return pnl_per_period.sum(dim=0)

def _calculate_correlation_matrix_functional(asset_returns_lookback_tensor, device_param=None):
    # derived_device = device_param if device_param is not None else asset_returns_lookback_tensor.device
    # derived_dtype = asset_returns_lookback_tensor.dtype

    # Removed empty/small tensor check
    corr_matrix = torch.corrcoef(asset_returns_lookback_tensor.T)
    # Removed torch.nan_to_num
    return corr_matrix

def _apply_clustering_algorithm_functional(correlation_matrix_tensor, num_clusters_to_form, cluster_method, device_param=None):
    derived_device = device_param if device_param is not None else correlation_matrix_tensor.device
    num_assets = correlation_matrix_tensor.shape[0]

    # Removed num_assets == 0 check
    effective_n_clusters = min(num_clusters_to_form, num_assets)
    # Removed effective_n_clusters <= 0 check (it will use original num_clusters_to_form if < min result or proceed if min result is <=0)


    if cluster_method == 'spectral_clustering':
        pos_corr = torch.clamp(correlation_matrix_tensor, min=0)
        neg_corr = torch.abs(torch.clamp(correlation_matrix_tensor, max=0))
        labels = spectral_clustering_laplacian(pos_corr, neg_corr, effective_n_clusters)
    else:
        raise ValueError(f"Unsupported cluster_method: {cluster_method}")
    return labels.to(derived_device)

def _define_clusters_and_centroids_functional(asset_returns_lookback_tensor, initial_n_clusters_config, cluster_method, device_param=None):
    derived_device = device_param if device_param is not None else asset_returns_lookback_tensor.device

    asset_corr_matrix = _calculate_correlation_matrix_functional(asset_returns_lookback_tensor, derived_device)
    # num_assets = asset_corr_matrix.shape[0] # Not strictly needed after removing checks

    cluster_definitions = {}
    cluster_names_ordered = []
    # actual_n_clusters_formed = 0 # Calculated later

    output_corr_matrix = asset_corr_matrix # Default, may be overwritten

    # Removed num_assets == 0 check

    labels = _apply_clustering_algorithm_functional(asset_corr_matrix, initial_n_clusters_config, cluster_method, derived_device)
    # def handmade_unique(x):
    #   sorted_x, _ = torch.sort(x)
    #   mask = (sorted_x[1:] - sorted_x[:-1])>0
    #   return sorted_x[1:]*mask
    # unique_labels = handmade_unique(labels).nonzero_static(size = )[0]
    # unique_labels = torch.nonzero(unique_labels, as_tuple=False).squeeze()
    # print(unique_labels)

    centroids_to_cat = []
    cluster_names = []

    for label_val_tensor in torch.arange(initial_n_clusters_config):
        cluster_name = label_val_tensor + 1
        cluster_names.append(cluster_name)
        asset_indices_in_cluster = (labels == label_val_tensor).nonzero_static(size = labels.shape[0])[0]

        centroid_ts = asset_returns_lookback_tensor[:, asset_indices_in_cluster].mean(dim=1)
        cluster_definitions[cluster_name] = {
            'asset_indices': asset_indices_in_cluster,
            'centroid_ts': centroid_ts
        }
        centroids_to_cat.append(centroid_ts.unsqueeze(0))

    # Removed 'if centroids_to_cat:' check
    all_centroids_stacked = torch.cat(centroids_to_cat, dim=0)
    # Removed shape check for all_centroids_stacked before corrcoef
    output_corr_matrix = torch.corrcoef(all_centroids_stacked)
    # Removed torch.nan_to_num

    return cluster_definitions, cluster_names, initial_n_clusters_config, output_corr_matrix

# --- REFACTORED FUNCTION for calculating weighted cluster returns ---
def _calculate_weighted_cluster_returns_functional(
    data_slice_for_returns_calc,
    data_slice_for_centroid_weights_calc,
    cluster_definitions,
    cluster_names_ordered,
    sigma_for_weights
):
    device = data_slice_for_returns_calc.device
    dtype_returns = data_slice_for_returns_calc.dtype
    dtype_weights_data = data_slice_for_centroid_weights_calc.dtype

    # Removed 'if not cluster_definitions:' and 'if not cluster_names_ordered:' checks

    T_current_period = data_slice_for_returns_calc.shape[0]
    cluster_returns_list = []

    for cluster_name in cluster_names_ordered:
        info = cluster_definitions[cluster_name] # Direct access, removed .get and None check

        asset_indices_in_cluster = info['asset_indices']
        centroid_lookback_ts = info['centroid_ts']

        current_cluster_period_returns = torch.zeros(T_current_period, device=device, dtype=dtype_returns) # Default

        # Removed 'if len(asset_indices_in_cluster) > 0:'
        asset_gaussian_weights = torch.zeros(len(asset_indices_in_cluster), device=device, dtype=dtype_weights_data) # Default

        # Removed 'if len(centroid_lookback_ts) > 0' part, kept alignment check
        if data_slice_for_centroid_weights_calc.shape[0] == len(centroid_lookback_ts):
            assets_in_cluster_returns_centroid_def_period = data_slice_for_centroid_weights_calc.index_select(1, asset_indices_in_cluster)
            centroid_lookback_ts_casted = centroid_lookback_ts.to(dtype_weights_data)

            squared_distances = torch.sum((assets_in_cluster_returns_centroid_def_period - centroid_lookback_ts_casted.unsqueeze(1))**2, dim=0)
            weights = torch.exp(-squared_distances / (2 * (sigma_for_weights**2))) # Removed + 1e-12
            asset_gaussian_weights = weights

        total_gaussian_weight_sum = asset_gaussian_weights.sum()
        assets_in_cluster_current_period = data_slice_for_returns_calc.index_select(1, asset_indices_in_cluster)

        normalized_weights = asset_gaussian_weights / total_gaussian_weight_sum # May produce NaN/inf if sum is 0
        current_cluster_period_returns = (assets_in_cluster_current_period * normalized_weights.unsqueeze(0)).sum(dim=1)

        # Removed T_current_period == 0 check for specific return shape

        cluster_returns_list.append(current_cluster_period_returns)

    # Removed 'if not cluster_returns_list:' check
    return torch.stack(cluster_returns_list, dim=1)

def _fit_var_and_forecast_functional(
    lookback_series_tensor,
    n_series_for_var_model,
    var_order, forecast_horizon, device_param=None
):
    # derived_device = device_param if device_param is not None else lookback_series_tensor.device # Not used
    model = VAR(var_order)

    model.fit(lookback_series_tensor) # Use the passed var_order

    # Convert forecast back to tensor and move to original device
    forecast_tensor_batched = model.forecast(lookback_series_tensor[-var_order:], forecast_horizon)

    return forecast_tensor_batched # (Horizon, N_series_var_model)

def _pad_tensor_cols_functional(tensor, target_cols, current_rows_for_empty_case):
    output_dtype = tensor.dtype
    # Removed tensor.numel() == 0 check and its specific return for (num_rows, target_cols)

    current_cols = tensor.shape[1]
    if current_cols == target_cols:
        return tensor
    elif current_cols < target_cols:
        padding_shape = (tensor.shape[0], target_cols - current_cols)
        # Removed handling for padding_shape[0] == 0
        padding = torch.zeros(padding_shape, device=tensor.device, dtype=output_dtype)
        return torch.cat([tensor, padding], dim=1)
    else: # current_cols > target_cols (truncate)
        return tensor[:, :target_cols]

# --- REFACTORED Main processing function for a single window ---
def single_window_iteration_processor_refactored(
    lookback_asset_returns_item,
    eval_asset_returns_item_padded,
    current_eval_len_for_window,
    window_idx_batch,
    initial_n_clusters_config_static,
    cluster_method_static,
    var_order_static,
    sigma_for_weights_static,
    max_eval_len_static,
    run_naive_var_comparison_static,
    num_assets_overall_static,
    device_static
):
    actual_eval_asset_returns_item = eval_asset_returns_item_padded

    cluster_defs, cluster_names_ord, actual_n_clust_formed, _ = _define_clusters_and_centroids_functional(
        lookback_asset_returns_item, initial_n_clusters_config_static, cluster_method_static, device_static
    )

    # Removed 'if cluster_names_ord:'
    lookback_cluster_returns = _calculate_weighted_cluster_returns_functional(
        data_slice_for_returns_calc=lookback_asset_returns_item,
        data_slice_for_centroid_weights_calc=lookback_asset_returns_item,
        cluster_definitions=cluster_defs,
        cluster_names_ordered=cluster_names_ord,
        sigma_for_weights=sigma_for_weights_static
    )

    # print(lookback_cluster_returns)

    # Removed 'if lookback_cluster_returns.numel() > 0 and lookback_cluster_returns.shape[1] > 0:'
    forecasted_cluster_returns_raw = _fit_var_and_forecast_functional(
        lookback_cluster_returns, actual_n_clust_formed, var_order_static,
        current_eval_len_for_window, device_static
    )


    # Removed 'if current_eval_len_for_window > 0:'
    true_eval_cluster_returns_raw = _calculate_weighted_cluster_returns_functional(
        data_slice_for_returns_calc=actual_eval_asset_returns_item,
        data_slice_for_centroid_weights_calc=lookback_asset_returns_item,
        cluster_definitions=cluster_defs,
        cluster_names_ordered=cluster_names_ord,
        sigma_for_weights=sigma_for_weights_static
    )

    pnl_c_tensor = calculate_pnl_torch(forecasted_cluster_returns_raw, true_eval_cluster_returns_raw)
    # Removed 'if pnl_c_tensor.numel() > 0:'
    pnl_c_val = pnl_c_tensor.mean().to(torch.float32)
    # Removed 'if torch.isnan(pnl_c_val):'

    pnl_n_val = torch.tensor(0.0, dtype=torch.float32, device=device_static)
    forecasted_naive_returns_raw = torch.tensor(0.0, dtype=torch.float32, device=device_static)
    true_eval_naive_returns_raw = torch.tensor(0.0, dtype=torch.float32, device=device_static)

    if run_naive_var_comparison_static:
        # Removed 'if lookback_asset_returns_item.numel() > 0 and lookback_asset_returns_item.shape[1] > 0 :'
        forecasted_naive_returns_raw = _fit_var_and_forecast_functional(
            lookback_asset_returns_item, num_assets_overall_static, var_order_static,
            current_eval_len_for_window, device_static
        )

        # Removed 'if current_eval_len_for_window > 0:'
        true_eval_naive_returns_raw = actual_eval_asset_returns_item

        pnl_n_tensor = calculate_pnl_torch(forecasted_naive_returns_raw, forecasted_naive_returns_raw)
        # Removed 'if pnl_n_tensor.numel() > 0:'
        pnl_n_val = pnl_n_tensor.mean().to(torch.float32)
        # Removed 'if torch.isnan(pnl_n_val):'

    return (pnl_c_val, forecasted_cluster_returns_raw, true_eval_cluster_returns_raw,
            pnl_n_val, forecasted_naive_returns_raw, true_eval_naive_returns_raw,
            window_idx_batch
           )

# --- REFACTORED Main sliding window function ---
def run_sliding_window_var_evaluation_vmap(
    asset_returns_tensor, initial_lookback_len, eval_len, n_clusters_config,
    cluster_method, var_order, sigma_intra_cluster, num_windows,
    device=None, store_sample_forecasts=True, run_naive_var_comparison=True
):
    # Removed 'if device is None:' device will be asset_returns_tensor.device if not passed
    effective_device = device if device is not None else asset_returns_tensor.device

    num_assets_overall = asset_returns_tensor.shape[1]
    tensor_total_len = asset_returns_tensor.shape[0]

    base_lookback_start_indices = torch.arange(num_windows, device=effective_device) * eval_len
    base_lookback_end_indices = base_lookback_start_indices + initial_lookback_len

    valid_mask_initial = (base_lookback_end_indices <= tensor_total_len) & \
                         (base_lookback_end_indices < tensor_total_len)

    filtered_lb_start_indices = base_lookback_start_indices[valid_mask_initial]
    filtered_lb_end_indices = base_lookback_end_indices[valid_mask_initial]
    original_window_indices_filtered = torch.arange(num_windows, device=effective_device)[valid_mask_initial]

    # Removed 'if len(filtered_lb_start_indices) == 0:' and early return

    num_potentially_valid_windows = len(filtered_lb_start_indices)
    current_eval_lens_for_filtered = torch.full((num_potentially_valid_windows,), eval_len, dtype=torch.long, device=effective_device)
    for i in range(num_potentially_valid_windows): # This loop might be empty if num_potentially_valid_windows is 0
        max_possible_eval_len = tensor_total_len - filtered_lb_end_indices[i]
        current_eval_lens_for_filtered[i] = min(eval_len, max_possible_eval_len)

    valid_mask_eval_len = current_eval_lens_for_filtered > 0

    final_lookback_start_indices = filtered_lb_start_indices[valid_mask_eval_len]
    final_lookback_end_indices = filtered_lb_end_indices[valid_mask_eval_len]
    final_current_eval_lens = current_eval_lens_for_filtered[valid_mask_eval_len]
    final_original_window_indices_batch = original_window_indices_filtered[valid_mask_eval_len]

    # Removed 'if len(final_lookback_start_indices) == 0:' and early return

    actual_num_windows_to_process = len(final_lookback_start_indices)

    batched_lookback_slices = []
    batched_eval_slices_padded = []

    # for _ in repeat_for_robustness:
    #   for p in lag_orders_to_test:
    #       for k in num_clusters_to_test:
    for s in range(actual_num_windows_to_process): # This loop might be empty
        start = final_lookback_start_indices[s]
        end = final_lookback_end_indices[s]

        lb_slice = asset_returns_tensor[start:end, :]
        batched_lookback_slices.append(lb_slice)

        eval_s = end
        current_L = final_current_eval_lens[s].item()
        eval_e = eval_s + current_L
        ev_slice_actual = asset_returns_tensor[eval_s:eval_e, :]

        # # Padding logic kept for vmap compatibility
        # if current_L < eval_len:
        #     padding = torch.zeros((eval_len - current_L, num_assets_overall),
        #                           dtype=asset_returns_tensor.dtype, device=effective_device)
        #     ev_slice_padded = torch.cat([ev_slice_actual, padding], dim=0)
        # elif current_L == eval_len:
        #     ev_slice_padded = ev_slice_actual
        # else:
        #     ev_slice_padded = ev_slice_actual[:eval_len,:]

        batched_eval_slices_padded.append(ev_slice_actual)

    # These stacks will error if their respective lists are empty (e.g., actual_num_windows_to_process is 0)
    final_batched_lookback_data = torch.stack(batched_lookback_slices)
    final_batched_eval_data_padded = torch.stack(batched_eval_slices_padded)


    vmapped_processor = func.vmap(
        single_window_iteration_processor_refactored,
        in_dims=(0, 0, None, 0,
                   None, None, None, None, None, None, None, None),
        out_dims=0, randomness='different'
    )

    all_pnl_c_vals, all_forecast_c, all_actual_c, \
    all_pnl_n_vals, all_forecast_n, all_actual_n, \
    processed_window_indices = vmapped_processor(
        final_batched_lookback_data,
        final_batched_eval_data_padded,
        eval_len,
        final_original_window_indices_batch,
        n_clusters_config,
        cluster_method,
        var_order,
        sigma_intra_cluster,
        eval_len,
        run_naive_var_comparison,
        num_assets_overall,
        effective_device
    )

    all_window_pnl_cluster_dicts = []
    all_window_pnl_naive_dicts = []

    for i in range(actual_num_windows_to_process): # This loop might be empty
        win_id = processed_window_indices[i].item()

        # Removed filter 'if all_pnl_c_vals[i].item() != 0.0 or all_forecast_c[i].abs().sum() > 1e-9:'
        all_window_pnl_cluster_dicts.append({
             'Window_ID': win_id, 'Avg_Window_PNL': all_pnl_c_vals[i].item(),
             'VAR_Order': var_order, 'Method': 'Clustered VAR'
         })

        if run_naive_var_comparison:
            # Removed filter for naive pnl/forecast values
            all_window_pnl_naive_dicts.append({
                'Window_ID': win_id, 'Avg_Window_PNL': all_pnl_n_vals[i].item(),
                'VAR_Order': var_order, 'Method': 'Naive VAR'
            })

    sample_forecast_c, sample_actual_c, sample_win_idx = None, None, -1
    # Removed 'and actual_num_windows_to_process > 0' from condition
    if store_sample_forecasts:
        sample_idx_in_batch = 0 # Will error if actual_num_windows_to_process is 0
        actual_eval_len_for_sample = final_current_eval_lens[sample_idx_in_batch].item()

        sample_forecast_c = all_forecast_c[sample_idx_in_batch, :actual_eval_len_for_sample, :].clone()
        sample_actual_c = all_actual_c[sample_idx_in_batch, :actual_eval_len_for_sample, :].clone()
        sample_win_idx = processed_window_indices[sample_idx_in_batch].item()

    return {
        'cluster_avg_pnl_list': all_window_pnl_cluster_dicts,
        'naive_avg_pnl_list': all_window_pnl_naive_dicts,
        'sample_forecast_cluster': sample_forecast_c,
        'sample_actual_cluster': sample_actual_c,
        'sample_window_idx_cluster': sample_win_idx,
        'var_order_for_sample': var_order
    }

"""## 1. Load and Clean Data"""

# file_path = r'OPCL_20000103_20201231.csv' # Path to your data file
file_path = r'/content/OPCL_20000103_20201231.csv'
df = pd.read_csv(file_path)

# Removed 'if not df.empty:' checks. Code will error if df is empty and operations are called.
df.set_index('ticker', inplace=True)
df.columns = pd.to_datetime(df.columns.str.lstrip('X'), format='%Y%m%d').strftime('%Y-%m-%d')
df_cleaned = df.dropna().transpose()
df_cleaned.index = pd.to_datetime(df_cleaned.index)
print("Data loaded and cleaned. Sample (first 5 rows/cols):")
# This print will error if df_cleaned is too small or empty.
print(df_cleaned.iloc[0:5,0:5])

print(f"Shape of the cleaned data: {df_cleaned.shape}")

numpy_array = df_cleaned.astype(float).values
historical_data_tensor = torch.from_numpy(numpy_array).float()
historical_data_tensor = historical_data_tensor.to(device)
# historical_data_tensor = torch.log1p(historical_data_tensor)
"""## 2. VAR Lag Grid Search and Evaluation"""

##################################################################### PARAMETERS #####################################################################
initial_lookback_len = 252
evaluation_len = 5
num_clusters_config = 20
cluster_method_config = 'spectral_clustering'
sigma_config = 0.5
num_windows_config = 25
var_orders_to_test_config = range(5,11,2)
####################################################################################################################################################

all_lags_combined_pnl_data = []
final_sample_details = {}

# Removed dummy data definition block for testing as it's a form of edge case / setup handling.
# The script now assumes 'historical_data_tensor' and 'device' are correctly defined.
# if 'historical_data_tensor' not in globals() or 'device' not in globals():
#    ...

# Removed 'if historical_data_tensor.numel() > 0:'
for var_order_val in tqdm(var_orders_to_test_config, desc="VAR Order Grid Search"):
    results = run_sliding_window_var_evaluation_vmap(
        asset_returns_tensor=historical_data_tensor, # Will error if not defined or empty and used inappropriately by functions
        initial_lookback_len=initial_lookback_len,
        eval_len=evaluation_len,
        n_clusters_config=num_clusters_config,
        cluster_method=cluster_method_config,
        var_order=var_order_val,
        sigma_intra_cluster=sigma_config,
        num_windows=num_windows_config,
        device=device,
        store_sample_forecasts=(var_order_val == var_orders_to_test_config[-1]),
        run_naive_var_comparison=False
    )
    all_lags_combined_pnl_data.extend(results['cluster_avg_pnl_list'])
    all_lags_combined_pnl_data.extend(results['naive_avg_pnl_list'])

    # Removed 'and results.get('sample_forecast_cluster') is not None'
    if (var_order_val == var_orders_to_test_config[-1]):
        final_sample_details = results # Will store even if sample is None

print("\n--- Grid Search Completed ---")
# Removed 'if all_lags_combined_pnl_data:'
agg_data = {}
methods_seen = set()
for record in all_lags_combined_pnl_data: # Will error if all_lags_combined_pnl_data is empty but accessed.
    methods_seen.add(record['Method'])
    key = (record['VAR_Order'], record['Method'])
    agg_data.setdefault(key, []).append(record['Avg_Window_PNL'])

sorted_methods = sorted(list(methods_seen))
# Removed 'if not sorted_methods:'
header = f"{'VAR_Order':<10} | " + " | ".join([f"{m:<15}" for m in sorted_methods])
print("\nAverage Window PNL per Lag Order and Method:")
print(header)
print("-" * len(header))

for var_order in sorted(list(set(r['VAR_Order'] for r in all_lags_combined_pnl_data))):
    row_str = f"{var_order:<10} | "
    for method in sorted_methods:
        pnl_list = agg_data.get((var_order, method), []) # Keep .get for safety or it could KeyError
        # Removed 'if pnl_list:' for calculating avg_pnl. Division by zero if len is 0.
        avg_pnl = sum(pnl_list) / len(pnl_list) if pnl_list else float('nan') # Retained minimal check to avoid div by zero for print
        row_str += f"{avg_pnl:<15.6f} | "
    print(row_str.strip().rsplit('|', 1)[0].strip())


# 3. Visualization
# Removed 'if all_lags_combined_pnl_data:'
# Removed 'try-except' block for plotting
# Filter for valid numeric PNLs for plotting to prevent seaborn/matplotlib errors. This is a practical necessity for plotting.
plot_data_list = [d for d in all_lags_combined_pnl_data if isinstance(d.get('Avg_Window_PNL'), (int, float)) and not np.isnan(d.get('Avg_Window_PNL'))]

# Removed 'if plot_data_list:'
plot_df_data = {
    'VAR_Order': [d['VAR_Order'] for d in plot_data_list],
    'Avg_Window_PNL': [d['Avg_Window_PNL'] for d in plot_data_list],
    'Method': [d['Method'] for d in plot_data_list]
}
plt.figure(figsize=(14, 8))
sns.boxplot(x='VAR_Order', y='Avg_Window_PNL', hue='Method', data=plot_df_data) # Will error if plot_df_data is empty/malformed
plt.title('Distribution of Avg Window PNL by VAR Lag & Method (PyTorch with vmap)')
plt.xlabel('VAR Lag Order'); plt.ylabel('Avg Window PNL')
plt.legend(title='Forecast Method'); plt.grid(True, axis='y', alpha=0.7)
plt.tight_layout(); plt.show()


# Removed 'if final_sample_details and final_sample_details.get('sample_forecast_cluster') is not None:'
# Direct access, will error if keys don't exist or values are None and methods are called.
fc = final_sample_details['sample_forecast_cluster'].cpu().numpy()
ac = final_sample_details['sample_actual_cluster'].cpu().numpy()
win_idx = final_sample_details['sample_window_idx_cluster']
var_ord_sample = final_sample_details['var_order_for_sample']

print(f"\n--- Plotting Sample: Window {win_idx + 1 if win_idx != -1 else 'N/A'}, Clustered VAR Lag {var_ord_sample} ---")
num_series_to_plot = min(3, fc.shape[1]) # fc.shape[1] will error if fc is None
# Removed 'if num_series_to_plot == 0:'

for i in range(num_series_to_plot):
    plt.figure(figsize=(14, 7))
    # Removed data/length validation before plotting. Plotting functions might error.
    sns.lineplot(x=range(len(ac[:, i])), y=ac[:, i], label=f'Actual Cluster {i+1}', marker='o')
    sns.lineplot(x=range(len(fc[:, i])), y=fc[:, i], label=f'Forecast Cluster {i+1}', marker='x', linestyle='--')
    plt.title(f'Prediction vs Actual for Cluster {i+1} (Win {win_idx+1 if win_idx != -1 else "N/A"}, VAR {var_ord_sample}, PyTorch with vmap)')
    plt.xlabel('Forecast Step'); plt.ylabel('Cluster Return')
    plt.legend(); plt.grid(True); plt.tight_layout(); plt.show()

Using device: cuda
Data loaded and cleaned. Sample (first 5 rows/cols):
ticker            AA       ABM       ABT       ADI       ADM
2000-01-03 -0.013042 -0.009188 -0.007117 -0.036071  0.000000
2000-01-04  0.010043  0.012346 -0.012786 -0.044261  0.005277
2000-01-05  0.047628 -0.006192  0.011111  0.014493 -0.015915
2000-01-06 -0.011713  0.000000  0.032553 -0.027719  0.010695
2000-01-07 -0.016118  0.003091  0.028573  0.033654  0.005249
Shape of the cleaned data: (5279, 143)


VAR Order Grid Search:   0%|          | 0/3 [00:11<?, ?it/s]


RuntimeError: vmap: It looks like you're calling .item() on a Tensor. We don't support vmap over calling .item() on a Tensor, please try to rewrite what you're doing with other operations. If error is occurring somewhere inside PyTorch internals, please file a bug report.

In [ ]:
import torch

class VAR:
    """
    Vector Autoregression (VAR) model estimated using OLS on Yule-Walker equations.
    This implementation does not use nn.Module.

    The VAR(p) model is defined as:
    y_t = c + A_1 y_{t-1} + ... + A_p y_{t-p} + u_t
    where y_t is a k-dimensional vector of variables,
    c is a k-dimensional intercept vector,
    A_i are k x k coefficient matrices,
    u_t is a k-dimensional white noise vector with covariance matrix Sigma_u.

    Estimation follows the Yule-Walker method:
    1. Estimate mean and autocovariance matrices (Gamma_j) from data.
    2. Solve the Yule-Walker equations for A_i coefficients.
       The system solved is effectively:
       Gamma_cap @ [A_1, ..., A_p].T = [Gamma_1, ..., Gamma_p].T (transposed)
       where Gamma_cap is a block matrix of autocovariances, and the solution
       for [A_1.T, ..., A_p.T].T is found using least squares (lstsq).
    3. Estimate intercept c.
    4. Estimate residual covariance Sigma_u.
    """

    def __init__(self, lag_order: int, include_intercept: bool = True):
        if lag_order < 1:
            raise ValueError("Lag order must be at least 1.")
        self.lag_order = lag_order
        self.include_intercept = include_intercept

        self.k_ = None  # Number of variables
        self.intercept_ = None  # Intercept vector c (1 x k)
        self.lag_coef_ = None  # Stacked A_i' matrices (kp x k): [A_1.T, A_2.T, ..., A_p.T].T
                               # So, A_i = self.lag_coef_[(i-1)*k : i*k, :].T
        self.sigma_u_ = None  # Residual covariance matrix (k x k)
        self.fitted_ = False
        self.mu_ = None # Mean of the series used for fitting

    def _compute_autocov(self, X_processed: torch.Tensor, lag: int) -> torch.Tensor:
        """
        Computes sample autocovariance matrix Gamma_lag.
        X_processed: (T x k) tensor, already mean-centered if intercept is included.
        lag: integer lag.
        Returns: (k x k) covariance matrix.
        Uses divisor T. For YW, sample autocovariances are often E[(y_t-mu)(y_{t-h}-mu)'].
        The divisor T is common for Gamma_0 in some contexts, and T-h for Gamma_h.
        Using T for all for consistency in constructing the YW system is one approach.
        Statsmodels VAR uses T-p for OLS and T for YW Gamma matrix elements.
        Here, we use T for simplicity, recognizing it's one common definition.
        """
        T, k = X_processed.shape
        if T <= lag:
             # This can happen if T is very small relative to p.
             # The calling fit method should check T > p.
             # For autocov computation specifically, if lag >= T, result is ill-defined or zero.
            return torch.zeros((k,k), device=X_processed.device, dtype=X_processed.dtype)


        if lag == 0:
            # Gamma_0 = (1/T) * sum_{t=1 to T} (x_t - mu)(x_t - mu)'
            return (X_processed.T @ X_processed) / T
        else:
            # Gamma_lag = (1/T) * sum_{t=lag+1 to T} (x_t - mu)(x_{t-lag} - mu)'
            # X_processed[lag:] corresponds to x_t
            # X_processed[:-lag] corresponds to x_{t-lag}
            # For (T x k) matrices, (A.T @ B) / N gives sum of (a_t' b_t) / N if a_t,b_t are columns
            # Here, X_processed[lag:] are rows for y_t, X_processed[:-lag] are rows for y_{t-lag}
            # We need sum (y_t - mu)' (y_{t-lag} - mu) if y are columns
            # (X_processed[lag:].T @ X_processed[:-lag]) gives sum_t (y_t-mu)(y_{t-lag}-mu)'
            # where (y_t-mu) is a column vector (k x 1)
            # No, it's sum_t (y_t-mu) (y_{t-lag}-mu)' where (y_t-mu) is taken as a column vector.
            # X_processed is (N_eff x k). X_processed[lag:].T is (k x (T-lag))
            # X_processed[:-lag] is ((T-lag) x k)
            # So (X_processed[lag:].T @ X_processed[:-lag]) is (k x k)
            return (X_processed[lag:].T @ X_processed[:-lag]) / T


    def fit(self, data: torch.Tensor):
        """
        Fits the VAR(p) model to the data using Yule-Walker equations.

        Args:
            data (torch.Tensor): Time series data of shape (T x k),
                                 where T is the number of observations and
                                 k is the number of variables.
        """
        T, k = data.shape
        if T <= self.lag_order:
            raise ValueError(f"Number of observations (T={T}) must be greater than lag order (p={self.lag_order}).")

        self.k_ = k
        p = self.lag_order

        device = data.device
        dtype = data.dtype

        if self.include_intercept:
            self.mu_ = data.mean(dim=0, keepdim=True)  # Shape (1 x k)
            X_processed = data - self.mu_
        else:
            self.mu_ = torch.zeros((1, k), device=device, dtype=dtype) # Assume mean zero
            X_processed = data # Data is used as is

        # 1. Estimate autocovariance matrices Gamma_j
        # We need Gamma_0, Gamma_1, ..., Gamma_p
        gammas = [self._compute_autocov(X_processed, j) for j in range(p + 1)]

        # 2. Solve Yule-Walker equations for lag coefficients (A_i)
        # System based on Lütkepohl (2005), "New Introduction to Multiple Time Series Analysis", p.70 eq (3.2.2)
        # [A_1, ..., A_p] = [Gamma_1, ..., Gamma_p] @ inv(Gamma_cap)
        # where Gamma_cap (kp x kp) is block matrix: Gamma_cap[r,c] = Gamma_{r-c} (using Gamma_{-s} = Gamma_s.T)
        # Let A_coeffs_stacked = [A_1.T, ..., A_p.T].T (kp x k)
        # Then (Gamma_cap).T @ A_coeffs_stacked = [Gamma_1.T, ..., Gamma_p.T].T

        # Construct Gamma_cap (Lütkepohl's Gamma, often denoted Psi or Big_Gamma in other texts)
        # Gamma_cap[block_row_idx, block_col_idx] = Gamma_{block_row_idx - block_col_idx}

        block_rows_list = []
        for r_idx in range(p):  # For each block row (0 to p-1)
            current_block_row_elements = []
            for c_idx in range(p):  # For each block in that row (0 to p-1)
                lag_val = r_idx - c_idx
                if lag_val == 0:
                    block_to_add = gammas[0]                            # Gamma_0
                elif lag_val > 0:
                    block_to_add = gammas[lag_val]                      # Gamma_{lag_val}
                else:  # lag_val < 0
                    block_to_add = gammas[abs(lag_val)].T               # Gamma_{-abs(lag_val)} = Gamma_{abs(lag_val)}.T
                current_block_row_elements.append(block_to_add)

            # Concatenate blocks horizontally for this block row
            # Each element is k x k, so result is k x (p*k)
            block_row = torch.cat(current_block_row_elements, dim=1)
            block_rows_list.append(block_row)

        # Concatenate block rows vertically
        # Each block_row is k x (p*k), stacking p of them gives (p*k) x (p*k)
        gamma_cap = torch.cat(block_rows_list, dim=0)

        # Construct RHS vector: [Gamma_1.T, ..., Gamma_p.T].T (stacked vertically)
        # Each Gamma_j.T is (k x k). Stacking p of them vertically results in (pk x k)
        rhs_stacked_T_list = [gammas[j+1].T for j in range(p)] # gammas[1].T, ..., gammas[p].T
        rhs_stacked_T = torch.cat(rhs_stacked_T_list, dim=0)


        # Solve (Gamma_cap).T @ A_coeffs_stacked = rhs_stacked_T for A_coeffs_stacked
        # A_coeffs_stacked is self.lag_coef_
        try:
            # Using lstsq aligns with "OLS on Yule-Walker" by finding the least squares solution.
            self.lag_coef_ = torch.linalg.lstsq(gamma_cap.T, rhs_stacked_T).solution
        except torch.linalg.LinAlgError as e:
            raise RuntimeError(f"Failed to solve Yule-Walker system using lstsq. Matrix may be singular. Original error: {e}")


        # 3. Estimate intercept c
        if self.include_intercept:
            # c = (I - A_1 - ... - A_p) @ mu
            A_sum = torch.zeros((k, k), device=device, dtype=dtype)
            if p > 0:
                for i in range(p):
                  A_i_plus_1_T = self.lag_coef_[i * k:(i + 1) * k, :] # This is A_{i+1}.T
                  A_sum = A_sum + A_i_plus_1_T.T # This is A_{i+1}

            identity_k = torch.eye(k, device=device, dtype=dtype)
            self.intercept_ = ((identity_k - A_sum) @ self.mu_.T).T # Shape (1 x k)
        else:
            self.intercept_ = torch.zeros((1, k), device=device, dtype=dtype)

        # 4. Estimate residual covariance Sigma_u
        # Sigma_u = Gamma_0 - Sum_{j=1 to p} A_j @ Gamma_j.T (Lütkepohl eq 2.1.19)
        # Note: Lütkepohl has Gamma_j in his formula, meaning E[y_t y_{t-j}'].
        # Our Gamma_j means E[y_t y_{t-j}']. So Gamma_j.T is E[y_{t-j} y_t'].
        # Lütkepohl (3.2.5) for Sigma_u: Gamma_0 - [A_1 ... A_p] @ [Gamma_1 ... Gamma_p].T
        # Which is Gamma_0 - sum A_i Gamma_i.T
        # This is equivalent to Gamma_0 - sum A_i Gamma_{-i} (using Gamma_{-i} = Gamma_i.T)

        self.sigma_u_ = gammas[0].clone()
        if p > 0:
            for i in range(p): # i from 0 to p-1
                # A_{i+1} corresponds to the i-th block in lag_coef_
                A_i_plus_1_T = self.lag_coef_[i * k:(i + 1) * k, :] # A_{i+1}.T
                A_i_plus_1 = A_i_plus_1_T.T # A_{i+1}

                # We need A_{i+1} @ Gamma_{i+1}.T
                self.sigma_u_ -= A_i_plus_1 @ gammas[i+1].T

        self.fitted_ = True

    def _check_fitted(self):
        if not self.fitted_:
            raise RuntimeError("Model has not been fitted yet. Call fit() first.")

    def predict(self, y_init_lags, n_ahead:int):
        """
        Predicts future values of the time series.

        Args:
            y_init_lags (torch.Tensor): Initial lag values of shape (L x k) where L >= p.
                                      These are the most recent p observations, ordered
                                      chronologically (oldest to newest).
                                      Example: If p=2, y_init_lags should be [y_{t-p}; ... ; y_{t-1}].
            n_ahead (int): Number of steps to predict ahead.

        Returns:
            torch.Tensor: Predicted values of shape (n_ahead x k).
        """
        self._check_fitted()
        if y_init_lags.shape[1] != self.k_:
            raise ValueError(
                f"Initial lags have {y_init_lags.shape[1]} variables, "
                f"but model was fitted on {self.k_} variables."
            )

        p = self.lag_order
        k = self.k_

        # At this point, p > 0 and n_ahead > 0.
        if y_init_lags.shape[0] < p:
            raise ValueError(
                f"Need at least {self.lag_order} initial lags for prediction, got {y_init_lags.shape[0]}."
            )

        current_lags = y_init_lags[-p:].clone() # Shape (p x k), [y_{t-(p-1)}, ..., y_{t}] (oldest to newest)
                                                # Or, using t as 'now': [y_{t-p+1}, ..., y_t]
                                                # Or, to match Lütkepohl: [y_{t-p}, ..., y_{t-1}] used to predict y_t
                                                # The indexing current_lags[-(j+1)] handles this correctly.

        predictions_list = []

        # Use _n_ahead_val in the range function
        for _ in range(n_ahead):
            pred_row = self.intercept_.clone()

            for j in range(p):
                lagged_y_val_row_vec = current_lags[-(j + 1), :].reshape(1, k)
                A_j_plus_1_T = self.lag_coef_[j * k:(j + 1) * k, :]
                A_j_plus_1 = A_j_plus_1_T.T
                pred_row += lagged_y_val_row_vec @ A_j_plus_1

            predictions_list.append(pred_row)

            if p > 1:
                current_lags = torch.roll(current_lags, shifts=-1, dims=0)
            current_lags[-1, :] = pred_row

        return torch.cat(predictions_list, dim=0)

    def forecast(self, y_init_lags: torch.Tensor, steps: int) -> torch.Tensor:
        return self.predict(y_init_lags, steps)

    def get_params(self) -> dict:
        """Returns the estimated parameters."""
        self._check_fitted()
        return {
            "intercept": self.intercept_.clone() if self.intercept_ is not None else None,
            "lag_coefficients_stacked_T": self.lag_coef_.clone() if self.lag_coef_ is not None else None, # Stacked A_i.T (kp x k)
            "sigma_u": self.sigma_u_.clone() if self.sigma_u_ is not None else None
        }

    def get_individual_A_matrices(self) -> list:
        """Returns a list of A_i matrices [A_1, A_2, ..., A_p]."""
        self._check_fitted()
        if self.lag_order == 0:
            return []

        A_matrices = []
        for i in range(self.lag_order):
            # self.lag_coef_ stores A_{idx+1}.T at block idx
            A_i_plus_1_T = self.lag_coef_[i * self.k_ : (i + 1) * self.k_, :]
            A_matrices.append(A_i_plus_1_T.T.clone()) # A_i = (A_i.T).T
        return A_matrices